<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Meta_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.fc(x)

def inner_loop(model, optimizer, data, target, inner_lr=0.01):
    model.train()
    optimizer.zero_grad()
    output = model(data)
    loss = ((output - target) ** 2).mean()
    loss.backward()
    optimizer.step()
    return loss.item()

def maml_train(model, optimizer, data, target, meta_lr=0.001, inner_steps=5):
    meta_optimizer = optim.Adam(model.parameters(), lr=meta_lr)
    model.train()
    for _ in range(inner_steps):
        loss = inner_loop(model, optimizer, data, target)
        meta_optimizer.zero_grad()
        meta_optimizer.step()
    return loss

model = SimpleNet(input_size=2, output_size=1)
optimizer = optim.SGD(model.parameters(), lr=0.01)
data = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
target = torch.tensor([[1], [0]], dtype=torch.float32)

for epoch in range(10):
    loss = maml_train(model, optimizer, data, target)
    print(f'Epoch {epoch + 1}, Loss: {loss}')